In [88]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import tensorflow as tf
import sqlite3
import pandas as pd 
from keras.models import load_model
import numpy as np
import pickle



# Datos

In [89]:
conn = sqlite3.connect('sr_gaming/datos/metacritics.db')

df_scores = pd.read_sql("""SELECT
    l.id_juego,
    l.rating,
    l.genero,
    l.distribuidor,
    l.desarrollador,
    l.fecha_lanzamiento,
    u.id_usuario,
    u.cant_reviews,
    u.score_promedio,
    u.ratio_positivo,
    u.ratio_negativo,
    u.ratio_mixto,          

    i.score
    FROM interacciones AS i
        JOIN juegos as l ON l.id_juego = i.id_juego
        JOIN usuarios as u ON u.id_usuario = i.id_usuario
    WHERE score > 0
""", conn)

df_scores.head()

,id_juego,rating,genero,distribuidor,desarrollador,fecha_lanzamiento,id_usuario,cant_reviews,score_promedio,ratio_positivo,ratio_negativo,ratio_mixto,score
0,baldurs-gate-3,M,Western RPG,Larian Studios Games,Larian Studios Games,2023-08-03,attack-of-the-fanboy,1204,78,66,1,33,100
1,baldurs-gate-3,M,Western RPG,Larian Studios Games,Larian Studios Games,2023-08-03,gamesradar,3857,70,45,9,46,100
2,baldurs-gate-3,M,Western RPG,Larian Studios Games,Larian Studios Games,2023-08-03,gamingtrend,5123,78,71,5,25,100
3,baldurs-gate-3,M,Western RPG,Larian Studios Games,Larian Studios Games,2023-08-03,ggrecon,225,77,63,5,32,100
4,baldurs-gate-3,M,Western RPG,Larian Studios Games,Larian Studios Games,2023-08-03,gfinity,324,76,61,5,34,100


Generación de mapping, categorización y normalización de datos

In [90]:
## user_id
unique_user_id = df_scores["id_usuario"].unique().tolist()
user_id_2_id_mapping = {t: i for i, t in enumerate(unique_user_id)}
id_2_user_id_mapping = {user_id_2_id_mapping[k]: k for k in user_id_2_id_mapping}
df_scores["id_usuario"] = df_scores["id_usuario"].apply(lambda x: user_id_2_id_mapping[x])

## item_id
unique_item_id = df_scores["id_juego"].unique().tolist()
item_id_2_id_mapping = {t: i for i, t in enumerate(unique_item_id)}
id_2_item_id_mapping = {item_id_2_id_mapping[k]: k for k in item_id_2_id_mapping}
df_scores["id_juego"] = df_scores["id_juego"].apply(lambda x: item_id_2_id_mapping[x])

categorical_cols = ["distribuidor", "desarrollador", "genero", "rating"]

mappings = {}

for col in categorical_cols:

    # 1) imputar NaN con categoría fija
    df_scores[col] = df_scores[col].fillna("__UNKNOWN__")
    
    df_scores[col] = df_scores[col].astype("category")
    
    # Diccionarios
    categories = df_scores[col].cat.categories

    col_map = {
        "value_2_id": {cat: i for i, cat in enumerate(categories)},
        "id_2_value": {i: cat for i, cat in enumerate(categories)},
    }

    mappings[col] = col_map
    
    # Aplicar los códigos
    df_scores[col] = df_scores[col].cat.codes

# ---- fecha_lanzamiento: pasar a número tipo AAAAMM ----
df_scores["fecha_lanzamiento"] = pd.to_datetime(
    df_scores["fecha_lanzamiento"], 
    format="%Y-%m-%d", 
    errors="coerce"
)

df_scores["fecha_lanzamiento"] = (
    df_scores["fecha_lanzamiento"]
        .dt.strftime("%Y%m")
        .astype(float)
)

# ---- columnas numéricas a normalizar (incluyendo fecha_lanzamiento) ----
numeric_cols = [
    "cant_reviews",
    "score_promedio",
    "ratio_positivo",
    "ratio_mixto",
    "ratio_negativo",
    "fecha_lanzamiento",
]

# en cuáles querés hacer fillna(0.0) antes de normalizar
fillna_zero_cols = [
    "score_promedio",
    "ratio_positivo",
    "ratio_mixto",
    "ratio_negativo",
    "fecha_lanzamiento",  # si también querés imputar 0 acá
]

# mantenemos norm_params si lo querés seguir usando internamente
mappings.setdefault("norm_params", {})

# NUEVO: definimos variables para los min/max que van en el mapping final
fecha_min = fecha_max = None
cant_reviews_min = cant_reviews_max = None
score_prom_min = score_prom_max = None
ratio_pos_min = ratio_pos_max = None
ratio_mix_min = ratio_mix_max = None
ratio_neg_min = ratio_neg_max = None

for col in numeric_cols:
    # 1) imputación opcional
    if col in fillna_zero_cols:
        df_scores[col] = df_scores[col].fillna(0.0)
    
    # 2) min y max (sobre los datos ya imputados)
    col_min = df_scores[col].min()
    col_max = df_scores[col].max()

    # 3) guardar en mappings[norm_params] (por compatibilidad)
    mappings["norm_params"][col] = {
        "min": float(col_min),
        "max": float(col_max),
    }

    # 3bis) NUEVO: asignar a las variables con los nombres que querés
    if col == "fecha_lanzamiento":
        fecha_min = float(col_min)
        fecha_max = float(col_max)
    elif col == "cant_reviews":
        cant_reviews_min = float(col_min)
        cant_reviews_max = float(col_max)
    elif col == "score_promedio":
        score_prom_min = float(col_min)
        score_prom_max = float(col_max)
    elif col == "ratio_positivo":
        ratio_pos_min = float(col_min)
        ratio_pos_max = float(col_max)
    elif col == "ratio_mixto":
        ratio_mix_min = float(col_min)
        ratio_mix_max = float(col_max)
    elif col == "ratio_negativo":
        ratio_neg_min = float(col_min)
        ratio_neg_max = float(col_max)

    # 4) normalización min-max
    if col_max > col_min:
        df_scores[col] = (df_scores[col] - col_min) / (col_max - col_min)
    else:
        # caso degenerado: toda la columna es constante
        df_scores[col] = 0.0

# NUEVO: construir los diccionarios de categóricas con los nombres pedidos
desarrollador_2_id_mapping = mappings["desarrollador"]["value_2_id"]
id_2_desarrollador_mapping = mappings["desarrollador"]["id_2_value"]

distribuidor_2_id_mapping = mappings["distribuidor"]["value_2_id"]
id_2_distribuidor_mapping = mappings["distribuidor"]["id_2_value"]

genero_2_id_mapping = mappings["genero"]["value_2_id"]
id_2_genero_mapping = mappings["genero"]["id_2_value"]

rating_2_id_mapping = mappings["rating"]["value_2_id"]
id_2_rating_mapping = mappings["rating"]["id_2_value"]

# NUEVO: mapping final con los nombres EXACTOS que querés
mappings = {
    "user_id_2_id": user_id_2_id_mapping,
    "item_id_2_id": item_id_2_id_mapping,
    "desarrollador_2_id": desarrollador_2_id_mapping,
    "distribuidor_2_id": distribuidor_2_id_mapping,
    "rating_2_id": rating_2_id_mapping,
    "genero_2_id": genero_2_id_mapping,
    "id_2_user_id": id_2_user_id_mapping,
    "id_2_item_id": id_2_item_id_mapping,
    "id_2_desarrollador": id_2_desarrollador_mapping,
    "id_2_distribuidor": id_2_distribuidor_mapping,
    "id_2_rating": id_2_rating_mapping,
    "id_2_genero": id_2_genero_mapping,
    "fecha_min": fecha_min,
    "fecha_max": fecha_max,
    "cant_reviews_min": cant_reviews_min,
    "cant_reviews_max": cant_reviews_max,
    "score_prom_min": score_prom_min, 
    "score_prom_max": score_prom_max,
    "ratio_pos_min": ratio_pos_min,
    "ratio_pos_max": ratio_pos_max,
    "ratio_mix_min": ratio_mix_min,
    "ratio_mix_max": ratio_mix_max,
    "ratio_neg_min": ratio_neg_min,
    "ratio_neg_max": ratio_neg_max,
}

with open("sr_gaming/datos/mappings.pkl", "wb") as f:
    pickle.dump(mappings, f)

# scores
df_scores["score"] = df_scores["score"] / 100.0

df_scores.head()


,id_juego,rating,genero,distribuidor,desarrollador,fecha_lanzamiento,id_usuario,cant_reviews,score_promedio,ratio_positivo,ratio_negativo,ratio_mixto,score
0,0,4,117,1009,2160,0.999012,0,0.059511,0.677419,0.632911,0.030303,0.475410,1.0
1,0,4,117,1009,2160,0.999012,1,0.193082,0.419355,0.367089,0.272727,0.688525,1.0
2,0,4,117,1009,2160,0.999012,2,0.256822,0.677419,0.696203,0.151515,0.344262,1.0
3,0,4,117,1009,2160,0.999012,3,0.010221,0.645161,0.594937,0.151515,0.459016,1.0
4,0,4,117,1009,2160,0.999012,4,0.015205,0.612903,0.569620,0.151515,0.491803,1.0


In [91]:
df_scores.distribuidor.unique()

array([1009,   19, 1232, ...,  689, 1399,  172],
      shape=(2096,), dtype=int16)

In [92]:
dataset_inputs = {
    "user_id": df_scores["id_usuario"].to_numpy(dtype="float32"),
    "item_id": df_scores["id_juego"].to_numpy(dtype="float32"),
    "desarrollador_id": df_scores["desarrollador"].to_numpy(dtype="float32"),
    "distribuidor_id": df_scores["distribuidor"].to_numpy(dtype="float32"),
    "rating_id": df_scores["rating"].to_numpy(dtype="float32"),
    "genero_id": df_scores["genero"].to_numpy(dtype="float32"),
    "fecha_lanzamiento": df_scores["fecha_lanzamiento"].to_numpy(dtype="float32"),
    "cant_reviews": df_scores["cant_reviews"].to_numpy(dtype="float32"),
    "score_promedio": df_scores["score_promedio"].to_numpy(dtype="float32"),
    "ratio_positivo": df_scores["ratio_positivo"].to_numpy(dtype="float32"),
    "ratio_mixto": df_scores["ratio_mixto"].to_numpy(dtype="float32"),
    "ratio_negativo": df_scores["ratio_negativo"].to_numpy(dtype="float32"),
    
}

dataset_outputs = df_scores['score']

dataset = tf.data.Dataset.from_tensor_slices((dataset_inputs, dataset_outputs))
dataset = dataset.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

cut = int(df_scores.shape[0] * 0.8)
train_dataset = dataset.take(cut).batch(1024).cache()
test_dataset = dataset.skip(df_scores.shape[0] - cut).batch(1024).cache()

## Una torre (versión funcional)

In [93]:

# usuarios
num_users = df_scores["id_usuario"].nunique() + 1
user_input = keras.layers.Input(shape=[1], name="user_id")
user_embedding = keras.layers.Embedding(num_users, min(num_users//2+1, 50))(user_input)
user_vec = keras.layers.Flatten()(user_embedding)

# items
num_items = df_scores["id_juego"].nunique() + 1
item_input = keras.layers.Input(shape=[1], name="item_id")
item_embedding = keras.layers.Embedding(num_items, min(num_items//2+1, 50))(item_input)
item_vec = keras.layers.Flatten()(item_embedding)

# desarrollador
num_desarrolladores = df_scores["desarrollador"].nunique() + 1
desarrollador_input = keras.layers.Input(shape=[1], name="desarrollador_id")
desarrollador_embedding = keras.layers.Embedding(num_desarrolladores, min(num_desarrolladores//2+1, 50))(desarrollador_input)
desarrollador_vec = keras.layers.Flatten()(desarrollador_embedding)

# distribuidor
num_distribuidores = df_scores["distribuidor"].nunique() + 1
distribuidor_input = keras.layers.Input(shape=[1], name="distribuidor_id")
distribuidor_embedding = keras.layers.Embedding(num_distribuidores, min(num_distribuidores//2+1, 50))(distribuidor_input)
distribuidor_vec = keras.layers.Flatten()(distribuidor_embedding)

# ratinges
num_ratinges = df_scores["rating"].nunique() + 1
rating_input = keras.layers.Input(shape=[1], name="rating_id")
rating_embedding = keras.layers.Embedding(num_ratinges, min(num_ratinges//2+1, 50))(rating_input)
rating_vec = keras.layers.Flatten()(rating_embedding)

# genero
num_generos = df_scores["genero"].nunique() + 1
genero_input = keras.layers.Input(shape=[1], name="genero_id")
genero_embedding = keras.layers.Embedding(num_generos, min(num_generos//2+1, 50))(genero_input)
genero_vec = keras.layers.Flatten()(genero_embedding)

# fecha_lanzamiento
fecha_lanzamiento_input = keras.layers.Input(shape=[1], name="fecha_lanzamiento")

# cant_reviews
cant_reviews_input = keras.layers.Input(shape=[1], name="cant_reviews")

# score_promedio
score_promedio_input = keras.layers.Input(shape=[1], name="score_promedio")

# ratio_positivo
ratio_positivo_input = keras.layers.Input(shape=[1], name="ratio_positivo")

# ratio_mixto
ratio_mixto_input = keras.layers.Input(shape=[1], name="ratio_mixto")

# ratio_negativo
ratio_negativo_input = keras.layers.Input(shape=[1], name="ratio_negativo")

conc = keras.layers.Concatenate()([
    user_vec, item_vec, desarrollador_vec, distribuidor_vec, rating_vec, genero_vec, fecha_lanzamiento_input, cant_reviews_input,
    score_promedio_input, ratio_positivo_input, ratio_mixto_input, ratio_negativo_input,
    ])

scores = keras.Sequential([
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(64, activation="relu"),

    keras.layers.Dense(1, activation='linear'),
])(conc)

In [94]:
model = keras.Model(inputs=[
    user_input, item_input, desarrollador_input, distribuidor_input, rating_input, genero_input, fecha_lanzamiento_input, cant_reviews_input,
    score_promedio_input, ratio_positivo_input, ratio_mixto_input, ratio_negativo_input
    ], outputs=scores)

model.compile(
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.RootMeanSquaredError()],
    optimizer=keras.optimizers.Adagrad(learning_rate=0.1) #hiperparámetro
)

# Detiene el entrenamiento cuando no hay mejora en loss en tres épocas consecutivas
es_callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# Graba el modelo cuando cambia val_root_mean_squared_error
#mc_callback = keras.callbacks.ModelCheckpoint(filepath="checkpoint.{val_root_mean_squared_error:.2f}.weights.h5", monitor="val_root_mean_squared_error", mode="min",  save_weights_only=True, save_best_only=True)
mc_callback = keras.callbacks.ModelCheckpoint(filepath="checkpoint.{val_root_mean_squared_error:.2f}.keras", monitor="val_root_mean_squared_error", mode="min",  save_weights_only=False, save_best_only=True)

history = model.fit(train_dataset,
                    validation_data=test_dataset,
                    validation_freq=1,
                    epochs=100,
                    batch_size=1024,
                    callbacks=[es_callback, mc_callback]
                    )

Epoch 1/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - loss: 0.0192 - root_mean_squared_error: 0.1387 - val_loss: 0.0270 - val_root_mean_squared_error: 0.1642
Epoch 2/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.0144 - root_mean_squared_error: 0.1201 - val_loss: 0.0263 - val_root_mean_squared_error: 0.1621
Epoch 3/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.0141 - root_mean_squared_error: 0.1189 - val_loss: 0.0258 - val_root_mean_squared_error: 0.1608
Epoch 4/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 0.0140 - root_mean_squared_error: 0.1183 - val_loss: 0.0256 - val_root_mean_squared_error: 0.1599
Epoch 5/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0139 - root_mean_squared_error: 0.1178 - val_loss: 0.0254 - val_root_mean_squared_error: 0.1593
Epoch 6/100
270/270 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.0138 - root_mean_squared_error: 0.1174 - val_loss: 0.0252 - val_root_mean_squared_error: 0.1586
Epoch 7/100
270/270 ━━━━━━━━━━━━━━━━━━━

Predicción Ejemplo


In [95]:
modelo = load_model("sr_gaming/datos/checkpoint.0.16.keras")


test_loss, test_rmse = modelo.evaluate(test_dataset, batch_size=1024)
print("Test RMSE en escala 0–100:", test_rmse * 100)


270/270 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0245 - root_mean_squared_error: 0.1564
Test RMSE en escala 0–100: 15.636640787124634


In [96]:
# Cargar mappings
import pickle
with open("sr_gaming/datos/mappings.pkl", "rb") as f:
    mappings = pickle.load(f)

user_id_2_id          = mappings["user_id_2_id"]
item_id_2_id          = mappings["item_id_2_id"]
desarrollador_2_id    = mappings["desarrollador_2_id"]
distribuidor_2_id     = mappings["distribuidor_2_id"]
rating_2_id           = mappings["rating_2_id"]
genero_2_id           = mappings["genero_2_id"]
cant_reviews_min = mappings["cant_reviews_min"]
cant_reviews_max = mappings["cant_reviews_max"]
score_prom_min = mappings["score_prom_min"]
score_prom_max = mappings["score_prom_max"]
ratio_pos_min = mappings["ratio_pos_min"]
ratio_pos_max = mappings["ratio_pos_max"]
ratio_mix_min = mappings["ratio_mix_min"]
ratio_mix_max = mappings["ratio_mix_max"]
ratio_neg_min = mappings["ratio_neg_min"]
ratio_neg_max = mappings["ratio_neg_max"]

# para fecha
fecha_min = mappings["fecha_min"]
fecha_max = mappings["fecha_max"]

#Ejemplo
id_usuario_real = '1up'

id_usuario_int = user_id_2_id_mapping[id_usuario_real]

print(f'ID de usuario: {id_usuario_int}') #id_usuario_int


ID de usuario: 113


In [97]:
conn = sqlite3.connect('sr_gaming/datos/metacritics.db')

# Obtiene los juegos que el usuario no ha puntuado, es decir, desconocidos
df_cand = pd.read_sql("""
    SELECT
        j.id_juego,
        j.desarrollador,
        j.distribuidor,
        j.rating,
        j.genero,
        j.fecha_lanzamiento,
        u.cant_reviews,
        u.score_promedio,
        u.ratio_positivo,
        u.ratio_mixto,
        u.ratio_negativo
    FROM juegos j
    JOIN usuarios u ON u.id_usuario = ?
    WHERE j.id_juego NOT IN (
        SELECT id_juego
        FROM interacciones
        WHERE id_usuario = ?
    )
""", conn, params=[id_usuario_real, id_usuario_real])

df_cand


,id_juego,desarrollador,distribuidor,rating,genero,fecha_lanzamiento,cant_reviews,score_promedio,ratio_positivo,ratio_mixto,ratio_negativo
0,baldurs-gate-3,Larian Studios Games,Larian Studios Games,M,Western RPG,2023-08-03,3527,69,51,35,14
1,goldeneye-007,Rare Ltd.,Nintendo,T,FPS,1997-08-25,3527,69,51,35,14
2,grand-theft-auto-iii,DMA Design,Rockstar Games,M,Open-World Action,2001-10-22,3527,69,51,35,14
3,grand-theft-auto-v,Rockstar North,Rockstar Games,M,Open-World Action,2014-11-18,3527,69,51,35,14
4,half-life-2,Valve Software,VU Games,M,FPS,2004-11-16,3527,69,51,35,14
...,...,...,...,...,...,...,...,...,...,...,...
11363,ride-to-hell-retribution,Eutechnyx,Deep Silver,M,Action Adventure,2013-06-25,3527,69,51,35,14
11364,spogs-racing,Pronto Games,D2C Games,E,Auto Racing,2008-07-07,3527,69,51,35,14
11365,stormgate,Frost Giant Studios,Frost Giant Studios,None,Command RTS,2025-08-05,3527,69,51,35,14
11366,vroom-in-the-night-sky,Poisoft,Poisoft,E,Biking,2017-04-05,3527,69,51,35,14


In [98]:
## Mapping cada variable categórica
# user_id interno (el mismo para todas las filas)
id_usuario_int = user_id_2_id[id_usuario_real]
df_cand["user_id_int"] = id_usuario_int

# item_id interno
df_cand["item_id_int"] = df_cand["id_juego"].map(item_id_2_id)

# desarrollador_id interno
df_cand["desarrollador_id_int"] = df_cand["desarrollador"].fillna( '__UNKNOWN__')
df_cand["desarrollador_id_int"] = df_cand["desarrollador_id_int"].map(desarrollador_2_id)

# distribuidor_id interno
df_cand["distribuidor_id_int"] = df_cand["distribuidor"].fillna( '__UNKNOWN__')
df_cand["distribuidor_id_int"] = df_cand["distribuidor_id_int"].map(distribuidor_2_id)

# rating_id interno
df_cand["rating_id_int"] = df_cand["rating"].fillna( '__UNKNOWN__')
df_cand["rating_id_int"] = df_cand["rating_id_int"].map(rating_2_id)

# genero_id interno
df_cand["genero_id_int"] = df_cand["genero"].fillna( '__UNKNOWN__')
df_cand["genero_id_int"] = df_cand["genero"].map(genero_2_id)


## Normalizar fecha y numéricas igual que en train
df_cand["fecha_lanzamiento"] = pd.to_datetime(
    df_cand["fecha_lanzamiento"], errors="coerce"
)
df_cand["fecha_aaaamm"] = df_cand["fecha_lanzamiento"].dt.strftime("%Y%m").astype(float)

df_cand["fecha_lanzamiento_norm"] = (
    (df_cand["fecha_aaaamm"] - fecha_min) / (fecha_max - fecha_min)
)

## Normalizar variables numéricas igual que en train


df_cand["cant_reviews_norm"] = (
    (df_cand["cant_reviews"] - cant_reviews_min)
    / (cant_reviews_max - cant_reviews_min)
)

df_cand["score_promedio_norm"] = (
    (df_cand["score_promedio"] - score_prom_min)
    / (score_prom_max - score_prom_min)
)

df_cand["ratio_positivo_norm"] = (
    (df_cand["ratio_positivo"] - ratio_pos_min)
    / (ratio_pos_max - ratio_pos_min)
)

df_cand["ratio_mixto_norm"] = (
    (df_cand["ratio_mixto"] - ratio_mix_min)
    / (ratio_mix_max - ratio_mix_min)
)

df_cand["ratio_negativo_norm"] = (
    (df_cand["ratio_negativo"] - ratio_neg_min)
    / (ratio_neg_max - ratio_neg_min)
)

df_cand.head()

,id_juego,desarrollador,distribuidor,rating,genero,fecha_lanzamiento,cant_reviews,score_promedio,ratio_positivo,ratio_mixto,...,distribuidor_id_int,rating_id_int,genero_id_int,fecha_aaaamm,fecha_lanzamiento_norm,cant_reviews_norm,score_promedio_norm,ratio_positivo_norm,ratio_mixto_norm,ratio_negativo_norm
0,baldurs-gate-3,Larian Studios Games,Larian Studios Games,M,Western RPG,2023-08-03,3527,69,51,35,...,1009,4,117,202308.0,0.999012,0.176468,0.387097,0.443038,0.508197,0.424242
1,goldeneye-007,Rare Ltd.,Nintendo,T,FPS,1997-08-25,3527,69,51,35,...,1232,6,38,199708.0,0.986173,0.176468,0.387097,0.443038,0.508197,0.424242
2,grand-theft-auto-iii,DMA Design,Rockstar Games,M,Open-World Action,2001-10-22,3527,69,51,35,...,1518,4,67,200110.0,0.988158,0.176468,0.387097,0.443038,0.508197,0.424242
3,grand-theft-auto-v,Rockstar North,Rockstar Games,M,Open-World Action,2014-11-18,3527,69,51,35,...,1518,4,67,201411.0,0.994583,0.176468,0.387097,0.443038,0.508197,0.424242
4,half-life-2,Valve Software,VU Games,M,FPS,2004-11-16,3527,69,51,35,...,1911,4,38,200411.0,0.989645,0.176468,0.387097,0.443038,0.508197,0.424242


In [105]:

    # --- 5) Armar input_dict AL ESTILO "lista de items" -------------

input_dict = {
        "user_id":        df_cand["user_id_int"].to_numpy(dtype="float32"),
        "item_id":        df_cand["item_id_int"].to_numpy(dtype="float32"),
        "desarrollador_id": df_cand["desarrollador_id_int"].to_numpy(dtype="float32"),
        "distribuidor_id":  df_cand["distribuidor_id_int"].to_numpy(dtype="float32"),
        "rating_id":        df_cand["rating_id_int"].to_numpy(dtype="float32"),
        "genero_id":        df_cand["genero_id_int"].to_numpy(dtype="float32"),
        "fecha_lanzamiento": df_cand["fecha_lanzamiento_norm"].to_numpy(dtype="float32"),
        "cant_reviews":      df_cand["cant_reviews_norm"].to_numpy(dtype="float32"),
        "score_promedio":    df_cand["score_promedio_norm"].to_numpy(dtype="float32"),
        "ratio_positivo":    df_cand["ratio_positivo_norm"].to_numpy(dtype="float32"),
        "ratio_mixto":       df_cand["ratio_mixto_norm"].to_numpy(dtype="float32"),
        "ratio_negativo":    df_cand["ratio_negativo_norm"].to_numpy(dtype="float32"),
    }

    # --- 6) Predecir para TODOS los juegos candidatos ---------------

preds = modelo.predict(input_dict, batch_size=1024).squeeze()
df_cand["pred_score"] = preds

# Top N recomendaciones
N = 9
df_top = df_cand.sort_values("pred_score", ascending=False).head(N)

# Mostrar id_juego real + score predicho
print(df_top[["id_juego", "pred_score"]])

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
                                               id_juego  pred_score
13               the-legend-of-zelda-breath-of-the-wild    0.717273
31             the-legend-of-zelda-tears-of-the-kingdom    0.717007
27    the-legend-of-zelda-a-link-to-the-past-four-sw...    0.715471
9                                 red-dead-redemption-2    0.713944
3317  the-legend-of-zelda-breath-of-the-wild-the-master    0.712717
32    the-legend-of-zelda-tears-of-the-kingdom-nintendo    0.712250
3                                    grand-theft-auto-v    0.712141
144                                      metroid-fusion    0.711019
33                   the-legend-of-zelda-the-wind-waker    0.710333


In [103]:
preds_train = model.predict(dataset_inputs, batch_size=1024).squeeze()

print("train min:", preds_train.min(), "max:", preds_train.max(), "std:", preds_train.std())


337/337 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
train min: 0.13033879 max: 1.0110232 std: 0.10904927


In [ ]:
df_juegos = df_cand.sort_values("pred_score", ascending=False)  

id_juegos = [i for i in df_juegos["id_juego"]]

id_juegos
    

['the-legend-of-zelda-breath-of-the-wild',
 'the-legend-of-zelda-tears-of-the-kingdom',
 'the-legend-of-zelda-a-link-to-the-past-four-swords',
 'red-dead-redemption-2',
 'the-legend-of-zelda-breath-of-the-wild-the-master',
 'the-legend-of-zelda-tears-of-the-kingdom-nintendo',
 'grand-theft-auto-v',
 'metroid-fusion',
 'the-legend-of-zelda-the-wind-waker',
 'new-super-mario-bros-u-deluxe',
 'pikmin-1-plus-2',
 'la-noire-the-vr-case-files',
 'the-legend-of-zelda-ocarina-of-time-master-quest-2002',
 'super-mario-maker-2',
 'the-legend-of-zelda-ocarina-of-time',
 'the-legend-of-zelda-breath-of-the-wild-the',
 'new-super-luigi-u',
 'super-mario-bros-wonder',
 'the-legend-of-zelda-breath-of-the-wild-nintendo',
 'mass-effect-2-lair-of-the-shadow-broker',
 'yoshis-story',
 'the-legend-of-zelda-a-link-between-worlds',
 'the-legend-of-zelda-triforce-heroes',
 'new-super-mario-bros-2',
 'star-fox-64',
 'the-legend-of-zelda-collectors-edition',
 'l-a-noire-reefer-madness',
 'grand-theft-auto-vice-